# RAG Agent with Hugging Face NVIDIA Models + NVIDIA AI Endpoints LLM

This notebook demonstrates how to build a RAG (Retrieval-Augmented Generation) agent using:
- **Hugging Face versions** of NVIDIA's Llama 3.2 models for embeddings and reranking
- **NVIDIA AI Endpoints** for the LLM (through build.nvidia.com)

## Models Used:
- **Embedding Model**: `nvidia/llama-3.2-nv-embedqa-1b-v2` (Hugging Face)
- **Reranking Model**: `nvidia/llama-3.2-nv-rerankqa-1b-v2` (Hugging Face)
- **LLM Model**: `nvidia/nvidia-nemotron-nano-9b-v2` (NVIDIA AI Endpoints)

## Architecture:
- Local Hugging Face models for embedding and reranking (better performance, privacy)
- NVIDIA AI Endpoints for LLM (managed service, latest models)
- Complete LangGraph agent with retrieval tools


## 1. Installation and Setup


In [2]:
import os
import getpass

os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")

In [3]:
import logging
import torch
import torch.nn.functional as F
from pathlib import Path
from typing import List, Dict, Any, Optional
import numpy as np

from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain.schema import Document
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.prebuilt import create_react_agent

# Set up logging
logging.basicConfig(level=logging.INFO)
_LOGGER = logging.getLogger(__name__)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check for NVIDIA API key
if "NVIDIA_API_KEY" not in os.environ:
    print("⚠️  Warning: NVIDIA_API_KEY not found in environment variables.")
    print("Please set your NVIDIA API key to use the LLM:")
    print("export NVIDIA_API_KEY='your_api_key_here'")
else:
    print("✅ NVIDIA API key found!")

Using device: cuda
✅ NVIDIA API key found!


## 2. Configuration


In [4]:
# Data Ingestion Configuration
DATA_DIR = Path("../data/it-knowledge-base")
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120

# Model Configuration
LLM_MODEL = "nvidia/nvidia-nemotron-nano-9b-v2"  # NVIDIA AI Endpoints
RETRIEVER_EMBEDDING_MODEL = "nvidia/llama-3.2-nv-embedqa-1b-v2"  # Hugging Face
RETRIEVER_RERANK_MODEL = "nvidia/llama-3.2-nv-rerankqa-1b-v2"  # Hugging Face

# Model parameters
MAX_LENGTH = 8192
EMBEDDING_DIM = 2048


## 3. Custom Hugging Face Embedding Class


In [5]:
class HuggingFaceNVIDIAEmbeddings(Embeddings):
    """Custom embedding class for NVIDIA Llama 3.2 embedding model from Hugging Face."""
    
    def __init__(self, model_name: str, device: str = "auto", trust_remote_code: bool = True):
        self.model_name = model_name
        self.device = device if device != "auto" else ("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"Loading embedding model: {model_name}")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
        
        # Load model - the key is to use trust_remote_code=True for custom NVIDIA models
        print("Loading model with trust_remote_code=True for custom NVIDIA architecture...")
        self.model = AutoModel.from_pretrained(
            model_name, 
            trust_remote_code=trust_remote_code,
        )
        
        self.model.to(self.device)
        self.model.eval()
        
        # Add padding token if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        print(f"✅ Embedding model loaded on {self.device}")
    
    def average_pool(self, last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
        """Average pooling with attention mask."""
        last_hidden_states_masked = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        embedding = last_hidden_states_masked.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
        embedding = F.normalize(embedding, dim=-1)
        return embedding
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents."""
        # Add document prefix as specified in the model documentation
        prefixed_texts = [f"passage: {text}" for text in texts]
        return self._embed_texts(prefixed_texts)
    
    def embed_query(self, text: str) -> List[float]:
        """Embed a single query."""
        # Add query prefix as specified in the model documentation
        prefixed_text = f"query: {text}"
        embeddings = self._embed_texts([prefixed_text])
        return embeddings[0]
    
    def _embed_texts(self, texts: List[str]) -> List[List[float]]:
        """Internal method to embed texts."""
        embeddings = []
        
        # Process in batches to handle memory efficiently
        batch_size = 8
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            
            # Tokenize
            batch_inputs = self.tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors="pt"
            ).to(self.device)
            
            # Generate embeddings
            with torch.no_grad():
                outputs = self.model(**batch_inputs)
                batch_embeddings = self.average_pool(
                    outputs.last_hidden_state, 
                    batch_inputs["attention_mask"]
                )
            
            # Convert to list and add to results
            embeddings.extend(batch_embeddings.cpu().numpy().tolist())
        
        return embeddings


## 4. Custom Hugging Face Reranker Class


In [6]:
class HuggingFaceNVIDIAReranker(BaseDocumentCompressor):
    """Custom reranker class for NVIDIA Llama 3.2 reranking model from Hugging Face."""
    
    def __init__(self, model_name: str, device: str = "auto", trust_remote_code: bool = True, top_k: int = 3, max_length: int = 512):
        # Initialize parent class first
        super().__init__()
        
        # Store configuration
        self._model_name = model_name
        self._device = device if device != "auto" else ("cuda" if torch.cuda.is_available() else "cpu")
        self._top_k = top_k
        self._max_length = max_length
        
        print(f"Loading reranking model: {model_name}")
        
        # Load tokenizer with proper configuration
        self._tokenizer = AutoTokenizer.from_pretrained(
            model_name, 
            trust_remote_code=trust_remote_code,
            padding_side="left"
        )
        if self._tokenizer.pad_token is None:
            self._tokenizer.pad_token = self._tokenizer.eos_token
        
        # Load model for sequence classification (reranking)
        model_kwargs = {
            "trust_remote_code": trust_remote_code,
        }
        
        from transformers import AutoModelForSequenceClassification
        self._model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            **model_kwargs
        ).eval()
        
        if self._model.config.pad_token_id is None:
            self._model.config.pad_token_id = self._tokenizer.eos_token_id
        
        self._model.to(self._device)
        
        print(f"✅ Reranking model loaded on {self._device}")
    
    def _prompt_template(self, query: str, passage: str) -> str:
        """Format query and passage with the proper prompt template."""
        return f"question:{query} \n \n passage:{passage}"
    
    def compress_documents(
        self,
        documents: List[Document],
        query: str,
        callbacks: Optional[Any] = None,
    ) -> List[Document]:
        """Compress documents by reranking them based on relevance to the query."""
        if not documents:
            return documents
        
        # Calculate relevance scores for all documents in batch
        scores = self._calculate_relevance_scores(query, [doc.page_content for doc in documents])
        
        # Pair documents with their scores
        doc_scores = list(zip(documents, scores))
        
        # Sort by relevance score (descending) and return top_k
        sorted_docs = sorted(doc_scores, key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in sorted_docs[:self._top_k]]
    
    def _calculate_relevance_scores(self, query: str, documents: List[str]) -> List[float]:
        """Calculate relevance scores for multiple documents efficiently."""
        # Apply prompt template to all query-document pairs
        texts = [self._prompt_template(query, doc) for doc in documents]
        
        # Tokenize all texts in batch
        batch_dict = self._tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=self._max_length,
        )
        
        # Move to device
        batch_dict = {k: v.to(self._device) for k, v in batch_dict.items()}
        
        # Generate relevance scores
        with torch.inference_mode():
            logits = self._model(**batch_dict).logits
            scores = logits.view(-1).cpu().tolist()
        
        return scores

## 5. Initialize Models


In [7]:
# Initialize the embedding model (Hugging Face)
embeddings = HuggingFaceNVIDIAEmbeddings(
    model_name=RETRIEVER_EMBEDDING_MODEL,
    device=device
)

# Initialize the reranker model (Hugging Face)
reranker = HuggingFaceNVIDIAReranker(
    model_name=RETRIEVER_RERANK_MODEL,
    device=device,
    top_k=3
)

# Initialize the LLM (NVIDIA AI Endpoints)
print(f"Initializing LLM: {LLM_MODEL}")
llm = ChatNVIDIA(model=LLM_MODEL, temperature=0.6, top_p=0.95, max_completion_tokens=8192)
print("✅ All models initialized successfully!")


Loading embedding model: nvidia/llama-3.2-nv-embedqa-1b-v2
Loading model with trust_remote_code=True for custom NVIDIA architecture...
✅ Embedding model loaded on cuda
Loading reranking model: nvidia/llama-3.2-nv-rerankqa-1b-v2


A new version of the following files was downloaded from https://huggingface.co/nvidia/llama-3.2-nv-rerankqa-1b-v2:
- llama_bidirectional_model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


✅ Reranking model loaded on cuda
Initializing LLM: nvidia/nvidia-nemotron-nano-9b-v2
✅ All models initialized successfully!


/home/chris/Code/NVIDIA/workshop-build-an-agent/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/_common.py:229: UserWarning: Found nvidia/nvidia-nemotron-nano-9b-v2 in available_models, but type is unknown and inference may fail.
  warnings.warn(


## 6. Data Loading and Processing


In [8]:
# Check if data directory exists
if not DATA_DIR.exists():
    print(f"Warning: Data directory {DATA_DIR} does not exist.")
    print("Creating sample documents for demonstration...")
    
    # Create sample IT knowledge base documents
    sample_docs = [
        Document(
            page_content="Password Policy: All employees must use passwords with at least 12 characters, including uppercase, lowercase, numbers, and special characters. Passwords must be changed every 90 days.",
            metadata={"source": "password_policy.txt"}
        ),
        Document(
            page_content="VPN Access: To connect to the company VPN, use the Cisco AnyConnect client. Download it from the IT portal and use your employee credentials. Contact IT support if you experience connection issues.",
            metadata={"source": "vpn_guide.txt"}
        ),
        Document(
            page_content="Email Setup: Configure your email client with IMAP settings: Server: mail.company.com, Port: 993, Security: SSL/TLS. For SMTP: Server: smtp.company.com, Port: 587, Security: STARTTLS.",
            metadata={"source": "email_setup.txt"}
        ),
        Document(
            page_content="Software Installation: All software installations require IT approval. Submit requests through the IT portal. Unauthorized software installation may result in security violations.",
            metadata={"source": "software_policy.txt"}
        ),
        Document(
            page_content="Backup Policy: All critical data must be backed up daily. Use the company-approved backup solution. Personal devices are not covered under the backup policy.",
            metadata={"source": "backup_policy.txt"}
        )
    ]
    docs = sample_docs
else:
    # Load documents from directory
    _LOGGER.info(f"Reading knowledge base data from {DATA_DIR}")
    data_loader = DirectoryLoader(
        DATA_DIR,
        glob="**/*",
        loader_cls=TextLoader,
        show_progress=True,
    )
    docs = data_loader.load()

print(f"Loaded {len(docs)} documents")


INFO:__main__:Reading knowledge base data from ../data/it-knowledge-base
100%|██████████| 12/12 [00:00<00:00, 3230.74it/s]

Loaded 12 documents


## 7. Create Vector Database and Retriever


In [9]:
# Split the data into chunks
_LOGGER.info(f"Splitting documents into chunks")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks")

# Create FAISS vector database
_LOGGER.info(f"Creating FAISS vector database with {len(chunks)} chunks")
vectordb = FAISS.from_documents(chunks, embeddings)
print("Vector database created successfully!")

# Create base retriever
kb_retriever = vectordb.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}
)

# Combine retriever with reranker
retriever = ContextualCompressionRetriever(
    base_retriever=kb_retriever,
    base_compressor=reranker,
)

print("Retriever with reranking created successfully!")


INFO:__main__:Splitting documents into chunks
INFO:__main__:Creating FAISS vector database with 155 chunks


Created 155 chunks


INFO:faiss.loader:Loading faiss with AVX512 support.
INFO:faiss.loader:Successfully loaded faiss with AVX512 support.


Vector database created successfully!
Retriever with reranking created successfully!


## 8. Create LangGraph Agent


In [10]:
# Create the retriever tool for agentic use
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="company_llc_it_knowledge_base",
    description=(
        "Search the internal IT knowledge base for Company LLC IT related questions and policies. "
        "Use this tool to find information about passwords, VPN, email setup, software policies, and other IT topics."
    ),
)

# Define the system prompt for the agent
SYSTEM_PROMPT = (
    "You are an IT help desk support agent.\n"
    "- Use the 'company_llc_it_knowledge_base' tool for questions likely covered by the internal IT knowledge base.\n"
    "- Always write grounded answers. If unsure, say you don't know.\n"
    "- If the knowledge base doesn't contain sufficient information, clearly state what information is missing.\n"
    "- Keep answers brief, to the point, and conversational."
)

# Create the ReAct agent
agent = create_react_agent(
    model=llm,
    tools=[retriever_tool],
    prompt=SYSTEM_PROMPT,
)

print("✅ LangGraph ReAct agent created successfully!")


✅ LangGraph ReAct agent created successfully!


/home/chris/Code/NVIDIA/workshop-build-an-agent/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:715: UserWarning: Model 'nvidia/nvidia-nemotron-nano-9b-v2' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(


## 9. Test the Agent


In [11]:
# Test the agent with sample queries
test_queries = [
    "What are the password requirements for employees?",
    "How do I set up VPN access?",
    "What are the email server settings?",
    "Can I install my own software?",
    "What's the backup policy?"
]

def test_agent(query: str):
    """Test the agent with a single query."""
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    try:
        # Invoke the agent
        response = agent.invoke({"messages": [("user", query)]})
        
        # Extract the final message
        final_message = response["messages"][-1].content
        print(f"Agent Response:\n{final_message}")
        
    except Exception as e:
        print(f"Error: {str(e)}")
    
    print("-" * 60)

# Test with the first query
test_agent(test_queries[0])



Query: What are the password requirements for employees?
Agent Response:
The password requirements for employees are as follows:  
- Minimum of **12 characters**.  
- Must include at least one uppercase letter, one lowercase letter, one number, and one special character.  
- Avoid common words, sequential numbers, or previously used passwords.  

Let me know if you need further clarification!
------------------------------------------------------------


## Summary

This notebook demonstrates a **hybrid approach** combining:

### 🏠 Local Hugging Face Models:
- **Embedding**: `nvidia/llama-3.2-nv-embedqa-1b-v2`
- **Reranking**: `nvidia/llama-3.2-nv-rerankqa-1b-v2`

### ☁️ NVIDIA AI Endpoints:
- **LLM**: `nvidia/nvidia-nemotron-nano-9b-v2`

### Key Benefits:

1. **Best of Both Worlds**:
   - Local models for embedding/reranking (privacy, performance, cost)
   - Managed LLM service (latest models, no infrastructure)

2. **Performance Optimizations**:
   - GPU acceleration for local models
   - Efficient batching and memory management
   - Proper query/passage prefixing as per model specs

3. **Production Ready**:
   - Complete LangGraph agent with tool integration
   - Proper error handling and logging
   - Interactive testing capabilities

### Architecture Flow:

1. **Document Processing**: Text splitting and chunking
2. **Embedding**: Local Hugging Face model generates embeddings
3. **Vector Search**: FAISS similarity search (k=6)
4. **Reranking**: Local Hugging Face model reranks to top 3
5. **Generation**: NVIDIA AI Endpoints LLM generates final response

### Usage:

- Run cells sequentially to set up the complete system
- Use `test_agent()` function for individual queries
- Use `chat_with_agent()` for interactive conversations
- Type 'test' in chat to run all predefined test queries

### Requirements:

- NVIDIA API key for LLM access
- GPU recommended for local models (falls back to CPU)
- Internet connection for model downloads and LLM API calls

The models used are:
- [nvidia/llama-3.2-nv-embedqa-1b-v2](https://huggingface.co/nvidia/llama-3.2-nv-embedqa-1b-v2)
- [nvidia/llama-3.2-nv-rerankqa-1b-v2](https://huggingface.co/nvidia/llama-3.2-nv-rerankqa-1b-v2)
